In [9]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RewardLM.git
# %cd RewardLM/
# !pip install -r requirements.txt
# from huggingface_hub import login
# login(token = '')  # https://huggingface.co/settings/tokens

In [1]:
# disable welcome message
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# 🥞 Reinforcement Learning with Automatic Feedback (RLAF)

In [2]:
import torch
from rewardlm.core.RL.RLModel import RLModel
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from transformers import GenerationConfig

rlmanager = RLModel(
    model_id = 'MBZUAI/LaMini-GPT-124M',
    reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target',
    optimized = True,   # use 8-bit PEFT
    bs = 16,
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
    generation_config=GenerationConfig(
        max_new_tokens = 40,
        min_new_tokens = 7,
        num_beams = 5,
        early_stopping = True,
        pad_token_id = 0,       # crashes while using batchsize > 1 only on mps device if not set
        temperature = 0.8,
        top_p = .8,
        # diversity_penalty = .1, # should use num_beam_groups > 1
    )
)


data = get_DIALOCONAN_prepro(delete_last_assistant_response = True)
dataset = rlmanager.generate_dataset(text = data)

history = rlmanager.train_PPO(dataset = dataset)

bin /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 589824 || all params 125030400 || trainable(%): 0.47
None


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:221: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(
0it [00:00, ?it/s]

0, 
 -len: 152-> "User: We’ve just imported 20k ticking time bombs from Afghanistan and we have homeless veterans getting arrested for begging in the street!.
Assistant:"
	 -len: 135-> "

We’ve just imported 20k ticking time bombs from Afghanistan and we have homeless veterans getting arrested for begging in the street."
---------

1, 
 -len: 357-> "User: We’ve just imported 20k ticking time bombs from Afghanistan and we have homeless veterans getting arrested for begging in the street!.
Assistant: Surely people who also risked their lives to help our forces also need us to honour our promises?
User: Our own must always come first. We have at least 13k homeless ex-soldiers sleeping rough.
Assistant:"
	 -len: 87-> "
Our own must always come first. We have at least 13k homeless veterans sleeping rough."
---------

2, 
 -len: 604-> "User: We’ve just imported 20k ticking time bombs from Afghanistan and we have homeless veterans getting arrested for begging in the street!.
Assistant: Surely

100%|██████████| 1/1 [00:02<00:00,  2.20s/it]
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
0it [00:40, ?it/s]


ValueError: Responses are too short. Make sure they are at least 4 tokens long.

# 👨🏼‍🏫 Model fine-tune

In [ ]:
import torch
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.data.CustomDatasets import PromptDataset_CLM


# select model
model_id = 'facebook/opt-350m'
generator_manager = GenerativeModel(
    model_id,
    load_dtype = '8-bit' if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

# download dataset
data = get_DIALOCONAN_prepro()

dataset = PromptDataset_CLM(
    tokenizer = generator_manager.tokenizer,
    text = data,
    custom_prompt = custom_prompt,
)

generator_manager.fine_tune(
    torch_dataset = dataset, 
    optimized = True if torch.cuda.is_available() else False,
)

# ⚖️ ToxicityMeter

In [ ]:
from transformers import GenerationConfig
from rewardlm.ToxicityMeter import ToxicityMeter
from rewardlm.utils.general_utils import device_selector

model_id = 'bigscience/bloomz-3b'
reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'
batchsize = 16
generation_config = GenerationConfig(
    max_new_tokens = 25,
    num_beams = 5,
    early_stopping = True,
    # crashes while using batchsize > 1 on mps device if not set:
    pad_token_id = 0,
    temperature = 0.8,
    top_p = .8,
    # diversity_penalty = .1, # should use num_beam_groups > 1
)

load_dtype = 'bf16' # can be ['8-bit', 'bf16', 'fp32']
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = load_dtype,
    toxicity_model_id = reward_model_id,
    device = device_selector(),
    generation_config = generation_config,
)

In [ ]:
from rewardlm.data.data_utils import get_real_toxicity_prompts

custom_prompt = ('Statement: "{prompt}".\nResponse: ')
# custom prompts required by the original paper of RedPajama
if model_id == 'togethercomputer/RedPajama-INCITE-Chat-3B-v1':
    custom_prompt = ('<human>: "{prompt}"\n<bot>: ')

toxicity_df = toxicity_meter.measure_toxicity(
    text_prompt = get_real_toxicity_prompts()['text'].to_list(),
    custom_prompt = custom_prompt, 
    generation_config = generation_config,
    batch_size = batchsize,
)

In [ ]:
# save csv in tmp folder
fld = './result analysis/tmp'
toxicity_df.to_csv(
    fld + f'/measured_tox_instruct_{model_id.split("/")[-1]}_{load_dtype}.csv'
)